In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from data_prep_func import bucket

In [2]:
column_names = [
    'Age',
    'Sex',
    'Job', # numeric: 0 — unskilled and non-resident, 1 — unskilled and resident, 2 — skilled, 3 — highly skilled
    'Housing',
    'Saving accounts',
    'Checking account',
    'Credit amount',
    'Duration', # in months
    'Purpose',
    'Risk'
]

In [3]:
used_columns = column_names
target = 'Risk'
sensitive = 'Sex'

# Reading data

In [4]:
original_data = pd.read_csv('/home/luiz/ufpb/mestrado/code/falsb/benchmark/data/german/german.csv', header=None)
data = pd.read_csv('/home/luiz/ufpb/mestrado/code/falsb/benchmark/data/german/simpler_german.csv', index_col=0)

In [5]:
data['Risk'] = original_data[20]
data['Risk'] = data['Risk'].map({1: 1, 2: 0}) # 1 = good -> 1, 2 = bad -> 0

In [6]:
data.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,1
1,22,female,2,own,little,moderate,5951,48,radio/TV,0
2,49,male,1,own,little,NaN,2096,12,education,1
3,45,male,2,free,little,little,7882,42,furniture/equipment,1
4,53,male,2,free,little,little,4870,24,car,0


In [7]:
data.tail()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
995,31,female,1,own,little,NaN,1736,12,furniture/equipment,1
996,40,male,3,own,little,little,3857,30,car,1
997,38,male,2,own,little,NaN,804,12,radio/TV,1
998,23,male,2,free,little,little,1845,45,radio/TV,0
999,27,male,2,own,moderate,moderate,4576,45,car,1


In [8]:
print(len(data.index))

1000


### Remove unused columns

In [9]:
data = pd.DataFrame(data, columns=used_columns)

# Treating missing values

In [10]:
data['Saving accounts'].value_counts()

little        603
moderate      103
quite rich     63
rich           48
Name: Saving accounts, dtype: int64

In [11]:
data['Saving accounts'] = data['Saving accounts'].fillna('none')

In [12]:
data['Saving accounts'].value_counts()

little        603
none          183
moderate      103
quite rich     63
rich           48
Name: Saving accounts, dtype: int64

In [13]:
data['Checking account'].value_counts()

little      274
moderate    269
rich         63
Name: Checking account, dtype: int64

In [14]:
data['Checking account'] = data['Checking account'].fillna('none')

In [15]:
data['Checking account'].value_counts()

none        394
little      274
moderate    269
rich         63
Name: Checking account, dtype: int64

In [16]:
print(len(data.index))

1000


# Normalizing continuous data

In [17]:
continous_attr = ['Credit amount', 'Duration']
scaler = MinMaxScaler()

In [18]:
data.describe()

,Age,Job,Credit amount,Duration,Risk
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,35.546000,1.904000,3271.258000,20.903000,0.700000
std,11.375469,0.653614,2822.736876,12.058814,0.458487
min,19.000000,0.000000,250.000000,4.000000,0.000000
25%,27.000000,2.000000,1365.500000,12.000000,0.000000
50%,33.000000,2.000000,2319.500000,18.000000,1.000000
75%,42.000000,2.000000,3972.250000,24.000000,1.000000
max,75.000000,3.000000,18424.000000,72.000000,1.000000


In [19]:
for attr in continous_attr:
    data[attr] = scaler.fit_transform(np.array(data[attr]).reshape(-1,1))

In [20]:
data.describe()

,Age,Job,Credit amount,Duration,Risk
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,35.546000,1.904000,0.166241,0.248574,0.700000
std,11.375469,0.653614,0.155317,0.177336,0.458487
min,19.000000,0.000000,0.000000,0.000000,0.000000
25%,27.000000,2.000000,0.061379,0.117647,0.000000
50%,33.000000,2.000000,0.113871,0.205882,1.000000
75%,42.000000,2.000000,0.204812,0.294118,1.000000
max,75.000000,3.000000,1.000000,1.000000,1.000000


# Age

In [21]:
data['Age'].describe()

count    1000.000000
mean       35.546000
std        11.375469
min        19.000000
25%        27.000000
50%        33.000000
75%        42.000000
max        75.000000
Name: Age, dtype: float64

In [22]:
data['Age'] = data['Age'].astype(int)

# Binarizing sex (risk is already binarized)

In [23]:
data['Sex'] = pd.get_dummies(data['Sex'])['female']

In [24]:
data.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,0,2,own,none,little,0.050567,0.029412,radio/TV,1
1,22,1,2,own,little,moderate,0.313690,0.647059,radio/TV,0
2,49,0,1,own,little,none,0.101574,0.117647,education,1
3,45,0,2,free,little,little,0.419941,0.558824,furniture/equipment,1
4,53,0,2,free,little,little,0.254209,0.294118,car,0


# One hot encoding categorical data

In [25]:
categorical_attr = ['Job', 'Housing', 'Saving accounts', 'Checking account', 'Purpose']

In [26]:
for attr in categorical_attr:
    column_idx = data.columns.get_loc(attr)
    data = pd.concat([data, pd.get_dummies(data[attr], prefix=attr)], axis=1)

In [27]:
data.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk,...,Checking account_none,Checking account_rich,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others
0,67,0,2,own,none,little,0.050567,0.029412,radio/TV,1,...,0,0,0,0,0,0,0,1,0,0
1,22,1,2,own,little,moderate,0.313690,0.647059,radio/TV,0,...,0,0,0,0,0,0,0,1,0,0
2,49,0,1,own,little,none,0.101574,0.117647,education,1,...,1,0,0,0,0,1,0,0,0,0
3,45,0,2,free,little,little,0.419941,0.558824,furniture/equipment,1,...,0,0,0,0,0,0,1,0,0,0
4,53,0,2,free,little,little,0.254209,0.294118,car,0,...,0,0,0,1,0,0,0,0,0,0


In [28]:
data.drop(categorical_attr, axis=1, inplace=True)

In [29]:
data.head()

,Age,Sex,Credit amount,Duration,Risk,Job_0,Job_1,Job_2,Job_3,Housing_free,...,Checking account_none,Checking account_rich,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others
0,67,0,0.050567,0.029412,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,22,1,0.313690,0.647059,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,49,0,0.101574,0.117647,1,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
3,45,0,0.419941,0.558824,1,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,0
4,53,0,0.254209,0.294118,0,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,0


In [30]:
data.columns

Index(['Age', 'Sex', 'Credit amount', 'Duration', 'Risk', 'Job_0', 'Job_1',
       'Job_2', 'Job_3', 'Housing_free', 'Housing_own', 'Housing_rent',
       'Saving accounts_little', 'Saving accounts_moderate',
       'Saving accounts_none', 'Saving accounts_quite rich',
       'Saving accounts_rich', 'Checking account_little',
       'Checking account_moderate', 'Checking account_none',
       'Checking account_rich', 'Purpose_business', 'Purpose_car',
       'Purpose_domestic appliances', 'Purpose_education',
       'Purpose_furniture/equipment', 'Purpose_radio/TV', 'Purpose_repairs',
       'Purpose_vacation/others'],
      dtype='object')

# Reordering the columns

In [31]:
columns_order = [
    'Age',
    'Credit amount',
    'Duration',
    'Job_0',
    'Job_1',
    'Job_2',
    'Job_3',
    'Housing_free',
    'Housing_own',
    'Housing_rent',
    'Saving accounts_little',
    'Saving accounts_moderate',
    'Saving accounts_none',
    'Saving accounts_quite rich',
    'Saving accounts_rich',
    'Checking account_little',
    'Checking account_moderate',
    'Checking account_none',
    'Checking account_rich',
    'Purpose_business',
    'Purpose_car',
    'Purpose_domestic appliances',
    'Purpose_education',
    'Purpose_furniture/equipment',
    'Purpose_radio/TV',
    'Purpose_repairs',
    'Purpose_vacation/others',
    'Sex',
    'Risk',
]

In [32]:
data.head()

,Age,Sex,Credit amount,Duration,Risk,Job_0,Job_1,Job_2,Job_3,Housing_free,...,Checking account_none,Checking account_rich,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others
0,67,0,0.050567,0.029412,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,22,1,0.313690,0.647059,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,49,0,0.101574,0.117647,1,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
3,45,0,0.419941,0.558824,1,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,0
4,53,0,0.254209,0.294118,0,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,0


In [33]:
data = data[columns_order]
data.head()

,Age,Credit amount,Duration,Job_0,Job_1,Job_2,Job_3,Housing_free,Housing_own,Housing_rent,...,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others,Sex,Risk
0,67,0.050567,0.029412,0,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
1,22,0.313690,0.647059,0,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
2,49,0.101574,0.117647,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,1
3,45,0.419941,0.558824,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1
4,53,0.254209,0.294118,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


# Saving data

In [34]:
print(len(data.index))

1000


In [35]:
data.to_csv('../data/german/post_prep/german.csv')